In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mph
import pprint

from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / 'util/'))

from functions import *
from materials import *

config = load_config( "config/Cavity_Qubit5G.yaml" )

In [3]:
client = mph.start()
pymodel = client.create('cavity')

In [4]:
parameters = pymodel/'parameters'
(parameters/'Parameters 1').rename('Parameters')

for parameter in config['parameter']:
    value = config['parameter'][parameter]
    pymodel.parameter( parameter, value)

pymodel.description('cavity_width', 'Cavity Width')
pymodel.description('cavity_length', 'Cavity Length')
pymodel.description('cavity_height', 'Cavity Height')
pymodel.description('cavity_fillet', 'Rounding Radius for cavity') 

In [5]:
## Java like
# model = pymodel.java
# model.component().create("comp1", True)
# model.component("comp1").geom().create("geom1", 3) # 3 means 3D
# model.component("comp1").geom("geom1").lengthUnit("\u00b5m")
# model.component("comp1").geom("geom1").create("wp1", "WorkPlane")
# model.component("comp1").geom("geom1").feature("wp1").set("unite", True)
# model.component("comp1").geom("geom1").feature("wp1").geom().create("r1", "Rectangle")
# model.component("comp1").geom("geom1").feature("wp1").geom().feature("r1").set("size", [2.4, 0.7])
# model.component("comp1").geom("geom1").feature("wp1").geom().feature("r1").set("pos", [-1.4, -0.35])
# model.component("comp1").geom("geom1").feature("wp1").geom().run("r1")

In [6]:


#
model = pymodel.java
comp1 = model.component().create("comp1", True)

geom1 = comp1.geom().create("geom1", 3) # 3 means 3D
geom1.lengthUnit("um")
geom1.run()

# Work plane 1
wp1 = geom1.create("wp1", "WorkPlane")
wp1.set("unite", True)
r1 = wp1.geom().create("r1", "Rectangle")
r1.set("size", ['cavity_width', 'cavity_length'])
r1.set("pos", ['-cavity_width/2', '0'])
wp1.geom().run("r1")

fil1 = wp1.geom().create("fil1", "Fillet")
fil1.selection("pointinsketch").set("r1", 1, 2, 3, 4)
fil1.set("radius", 'cavity_fillet')
wp1.geom().run("fil1")

wp1.label("cav-def")
geom1.run("wp1")

# Extrude 1
ext1 = geom1.create("ext1", "Extrude")
ext1.set("workplane", "wp1")
ext1.selection("input").set("wp1")
ext1.setIndex("distance", 'cavity_height', 0) # 0 means first row ?

geom1.run("ext1")

# Work plane 2
wp2 = geom1.create("wp2", "WorkPlane")
wp2.set("unite", True)
wp2.label("Ports-def")
wp2.set("quickplane", "xz")
c1 = wp2.geom().create("c1", "Circle")
c1.set("r", 'port_outer_radius')
c1.set("pos", ['0', '(cavity_height + port_distance)/2'] ) # Both x, y values must be int/float or string

mir1 = wp2.geom().create("mir1", "Mirror")
mir1.selection("input").set("c1")
mir1.set("pos", ['0', 'cavity_height/2'])
mir1.set("axis", [0., 1.])
mir1.set("keep", True)
wp2.geom().run("mir1")
geom1.run('wp2')

# Extrude 2

ext2 = geom1.create("ext2", "Extrude")
ext2.set("workplane", "wp2")
ext2.selection("input").set("wp2")
ext2.setIndex("distance", "port_height", 0)
ext2.label("Ports")

geom1.run("ext2")

# Work plane 3
wp3 = geom1.create("wp3", "WorkPlane")
wp3.set("unite", True)
wp3.set("quickplane", "xz")
wp3.set("quicky", "-port_height")
c2 = wp3.geom().create("c2", "Circle")
c2.set("pos",  ['0', '(cavity_height + port_distance)/2'] )
c2.set("r", 'port_inner_radius')
wp3.label("Pin-def")
geom1.run('wp3')

# Extrude 3
ext3 = geom1.create("ext3", "Extrude")
ext3.set("workplane", "wp3")
ext3.selection("input").set("wp3")
ext3.setIndex("distance", "-port_in_depth", 0)
ext3.label("Pin")
geom1.run("ext3")

# Work plane 4
wp4 = geom1.create("wp4", "WorkPlane")
wp4.set("unite", True)
wp4.set("quickplane", "xz")
wp4.set("quicky", "-port_height")
c3 = wp4.geom().create("c3", "Circle")
c3.set("pos",  ['0', '(cavity_height - port_distance)/2'] )
c3.set("r", 'port_inner_radius')
wp4.label("Pout-def")
geom1.run('wp4')

# Extrude 4
ext4 = geom1.create("ext4", "Extrude")
ext4.set("workplane", "wp4")
ext4.selection("input").set("wp4")
ext4.setIndex("distance", "-port_out_depth", 0)
ext4.label("Pout")
geom1.run("ext4")

# Work plane 5
wp5 = geom1.create("wp5", "WorkPlane")
wp5.set("unite", True)
wp5.set("quickz", 'cavity_height/2')
r2 = wp5.geom().create("r2", "Rectangle")
r2.set("size", ['wafer_space_width', 'wafer_space_length'])
r2.set("base", "center")
r2.set("pos", ['0', 'cavity_length/2'])
wp5.geom().run("r2")

fil2 = wp5.geom().create("fil2", "Fillet")
fil2.selection("pointinsketch").set("r2", 4, 3, 1, 2)
fil2.set("radius", 'wafer_space_fillet')
wp5.geom().run("fil2")

wp5.label("Chip space-def")
geom1.run("wp5")

# Extrude 5
ext5 = geom1.create("ext5", "Extrude")
ext5.set("workplane", "wp5")
ext5.selection("input").set("wp5")
ext5.setIndex("distance", '-wafer_space_height', 0)
ext5.label("Chip space")
geom1.run("ext5")

# Union 1
uni1 = geom1.create("uni1", "Union")
uni1.selection("input").set("ext1", "ext2", "ext5")
uni1.set("intbnd", False)
geom1.run("uni1")

# Work plane 6
wp6 = geom1.create("wp6", "WorkPlane")
wp6.set("unite", True)
wp6.set("quickz", "cavity_height/2 - wafer_space_height")
r3 = wp6.geom().create("r3", "Rectangle")
r3.set("base", "center")
r3.set("pos", ['0', 'cavity_length/2'])
r3.set("size", ['wafer_width', 'wafer_length'])
wp6.geom().run("r3")
wp6.label("Chip")
geom1.run("wp6")

# Extrude 6
ext6 = geom1.create("ext6", "Extrude")
ext6.set("workplane", "wp6")
ext6.selection("input").set("wp6")
ext6.setIndex("distance", 'wafer_height', 0)
geom1.run("ext6")

# Work plane 7
wp7 = geom1.create("wp7", "WorkPlane")
wp7.set("unite", True)
wp7.set("quickz", "cavity_height/2 - wafer_space_height + wafer_height")
r4 = wp7.geom().create("r4", "Rectangle")
r4.set("size", ['chip_width', 'chip_height'])
r4.set("base", "center")
r4.set("pos", ['0', 'cavity_length/2 + chip_height/2 + chip_gap/2'])
wp7.geom().run("r4")

fil3 = wp7.geom().create("fil3", "Fillet")
fil3.selection("pointinsketch").set("r4", 1, 2, 3, 4)
fil3.set("radius", 'chip_fillet')
wp7.geom().run("fil3")

r5 = wp7.geom().create("r5", "Rectangle")
r5.set("base", "center")
r5.set("pos", ['0', 'cavity_length/2 + jj_height/2 + finger_height/2'])
r5.set("size", ["jj_width", "finger_height"])
wp7.geom().run("r5")

# Union 2
uni2 = wp7.geom().create("uni2", "Union")
uni2.selection("input").set("fil3", "r5")
uni2.set("intbnd", False)
wp7.geom().run("uni2")

mir2 = wp7.geom().create("mir2", "Mirror")
mir2.selection("input").set("uni2")
mir2.set("pos", ['0', 'cavity_length/2'])
mir2.set("keep", True)
mir2.set("axis", [0., 1.])
wp7.geom().run("mir2")
wp7.label("Transmon pad")

r6 = wp7.geom().create("r6", "Rectangle")
r6.set("base", "center")
r6.set("size", ['jj_width','jj_height'])
r6.set("pos", ['0', 'cavity_length/2'])
wp7.geom().run("r6")

geom1.run("wp7")


# run geometry and set view after making everything
geom1.run()
comp1.view("view1").set("transparency", True)



<java object 'com.comsol.clientapi.impl.ViewClient'>

In [7]:
assign_material(node = comp1, material = "air", tag = "mat1", name = "Air", selection = [1, 3, 4])
assign_material(node = comp1, material = "silicon", tag = "mat2", name = "Silicon", selection = 2)

In [8]:
mesh1 = comp1.mesh().create("mesh1")
emw  = comp1.physics().create("emw", "ElectromagneticWaves", "geom1")

lelement1 = emw.create("lelement1", "LumpedElement", 2)
lelement1.label("Zin")
lelement1.selection().set(13)
lelement1.set("PortType", "Coaxial")
lelement2 = emw.create("lelement2", "LumpedElement", 2)
lelement2.label("Zout")
lelement2.selection().set(10)
lelement2.set("PortType", "Coaxial")

lelement3 = emw.create("lelement3", "LumpedElement", 2)
lelement3.selection().set(43)
lelement3.set("LumpedElementType", "Inductor")
lelement3.set("Lelement", "chip_inductance")
lelement3.label("LJ")

<java object 'com.comsol.clientapi.physics.impl.PhysicsFeatureClient'>

In [9]:
std1 = model.study().create("std1")
eig = std1.create("eig", "Eigenfrequency")
eig.set("conrad", "1")
eig.set("linpsolnum", "auto")
eig.set("solnum", "auto")
eig.set("notsolnum", "auto")
eig.set("ngenAUX", "1")
eig.set("goalngenAUX", "1")
eig.setSolveFor("/physics/emw", True)

True

In [10]:
pymodel.save('mph/cavity_demo.mph')
client.remove(pymodel)